In [ ]:
import weaviate
import json
import pprint

print("Let's a go!")

print("Connecting to weaviate instance on localhost:8080...")
client = weaviate.Client("http://localhost:8080")
print("Client created")

pprint.pprint(client.schema.get())

result = (
    client.query
    .get("DriftBenchmark", ["prompt_token_len", "prompt_sent_len", "prompt", "completion_token_len", "completion_sent_len", "completion", "date_time", "chat_id"])
    .do()
)
print(json.dumps(result, indent=4))

In [ ]:
import weaviate
import json

print("Let's a go!")

print("Connecting to weaviate instance on localhost:8080...")
client = weaviate.Client("http://localhost:8080")
print("Client created")

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
print("sbert loaded")

def process_query(vec):
    where_filter = {
        "path": ["chat_id"],
        "operator": "Equal",
        "valueText": "c5f83121-758e-46b2-99d1-5ca0c6f59919",
    }
    nearVector = {"vector": vec}
    res = client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion", "_additional {certainty, distance}"]).with_near_vector(nearVector).with_where(where_filter).do()
    print("------------------------------------------------------------------------------------------------")
    print("-----------------------------------Most similar text -------------------------------------------")
    print("ChatID: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['chat_id'], end='')
    print(", Prompt Token Count: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['prompt_token_len'], end='')
    print(", Prompt Sentences Count: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['prompt_sent_len'], end='')
    print(", Prompt: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['prompt'], end='')
    print(", Completion Token Count: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['prompt_token_len'], end='')
    print(", Completion Sentences Count: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['prompt_sent_len'], end='')
    print(", Completion: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['completion'], end='')
    print(", Certainty: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['_additional']['certainty'])
    print(", Distance: ", end='')
    print(res['data']['Get']['DriftBenchmark'][0]['_additional']['distance'])
    print("------------------------------------------------------------------------------------------------")

print("------------------------------------------------------------------------------------------------")
print("Running a test query first...")

# this is exactly the text in cmpletion for 
#{
#"prompt": "how to create a custom shell using click and cmd",
#"completion": "To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific commands that you want to",
#"chat_id": "c5f83121-758e-46b2-99d1-5ca0c6f59919"
#}

test_query = "To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific commands that you want to"
query_vec = sbert_model.encode(test_query)
print(query_vec)
process_query(query_vec)
print("------------------------------------------------------------------------------------------------")
# Certainty: 0.9343259930610657

# now shorten it by 20 chars
test_query = "To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd ``` 3. Create a new class that inherits from the Cmd class and override the necessary methods to handle user input and commands. For example, you can override the `do_command` method to handle specific comma"
query_vec = sbert_model.encode(test_query)
print(query_vec)
process_query(query_vec)
print("------------------------------------------------------------------------------------------------")
# 1 Certainty: 0.9343259930610657
# 2 Certainty: 0.9157571196556091
#  difference: 0.0185688734054566

# 1 Distance: 0.13134801
# 2 Distance: 0.16848576
# difference: 0.03713775

# while True:
    
#     query = input("Enter query:")
#     if query=='q':
#         break

#     query_vec = sbert_model.encode(query)
#     #print(query_vec)
#     process_query(query_vec)

# Query with Cursor

In [ ]:
import weaviate
import json

print("Let's a go!")

print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

limit = 10
result_count = 0
aggregate_count = 0
batch_size = 5
class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

def get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        # print(json.dumps(res, indent=4))
        print("{}. {}".format(result_count, res['completion']))
        # res["data"]["Get"]
        result_count += 1
        
    if(aggregate_count >= limit):
        break

    # Update the cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]


In [ ]:
# 2. Run a weaviate query with a cursor, 
# one at a time, we send the benchmark prompt to cohere
# and just print the completion response 

import weaviate
import cohere
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('COHERE_API_KEY')

def IsOkString(value):
    if not value or len(value)==0:
        return False 
    else:
        return True

print("Let's a go!")

print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

# These are guardrails to prevent offending the free cohere API.
limit = 3
aggregate_count = 6
batch_size = 7

result_count = 0
class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

print("Creating cohere_client")
cohere_client = cohere.Client(api_key) # This is your trial API key

# you can add filters to this query - look in dev_nb\03_weaviate_queries_dev.ipynb

def wv_wv_get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()


print("Running the query...")
print_runtime()

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = wv_get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        # print(json.dumps(res, indent=4))
        
        the_prompt = res['prompt']
        print("\n  Weaviate Benchmark sample #{}.\n    Prompt: {}\n    Completion: {}".format(result_count, res['prompt'], res['completion']))
        result_count += 1
        
        response = cohere_client.generate(
            model='command',
            prompt=the_prompt,
            max_tokens=300,
            temperature=0.9,
            k=0,
            stop_sequences=[],
            return_likelihoods='NONE')
        
        # print the full response object:
        # print("Cohere prompt response: {}".format(response))
        
        prompt_response = response.generations[0].text.replace("\n", " ")
        print("\n  Cohere completion:\n    {}".format(prompt_response))        
        print("------------------------------------------------------------------------------------------------")
    
    if(aggregate_count >= limit):
        break

    # Update the cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]

## Comparing Sentence Similarities

The sentences (texts) are mapped such that sentences with similar meanings are close in vector space. One common method to measure the similarity in vector space is to use [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). For two sentences, this can be done like this:

Note: the model we'll (multi-qa-MiniLM-L6-cos-v1) use must be the same one we used to create the original DriftBenchmark import.

```python
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

#Sentences are encoded by calling model.encode()
emb1 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd")

emb2 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal.")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

With `SentenceTransformer('all-MiniLM-L6-v2')` we define which sentence transformer model we like to load. In this example, we load *all-MiniLM-L6-v2*, which is a MiniLM model fine tuned on a large dataset of over 1 billion training pairs.

BERT (and other transformer networks) output for each token in our input text an embedding. In order to create a fixed-sized sentence embedding out of this, the model applies mean pooling, i.e., the output embeddings for all tokens are averaged to yield a fixed-sized vector.

## Comparing Sentence Similarities

The sentences (texts) are mapped such that sentences with similar meanings are close in vector space. One common method to measure the similarity in vector space is to use [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). For two sentences, this can be done like this:

Note: the model we'll (multi-qa-MiniLM-L6-cos-v1) use must be the same one we used to create the original DriftBenchmark import.

```python
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

#Sentences are encoded by calling model.encode()
emb1 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal. 2. Import the necessary modules from Click and Cmd: ``` from click import Command, option, Argument from cmd import Cmd")

emb2 = model.encode("To create a custom shell using Click and Cmd, you can follow these steps:  1. Install the Click library by running `pip install click` in your terminal.")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)
```

In [ ]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

#Sentences are encoded by calling model.encode()
vector1 = model.encode("The pizza is hot")

# the result of model.encode is a 384-dimension vector:
# print(vector1)

# identical sentences produce cosine similarity of 1.0
emb1 = model.encode("The large dog is red")
emb2 = model.encode("The large dog is red")
cos_sim = util.cos_sim(emb1, emb2)
print("1 - Cosine-Similarity:", cos_sim.item())

# make a change that doesn't change the meaning too much
# and the the cosine similarity decreases a small amount.
emb1 = model.encode("The large dog is red")
emb2 = model.encode("The large canine is red")
cos_sim = util.cos_sim(emb1, emb2)
print("2 - Cosine-Similarity:", cos_sim.item())

# continue making small changes and watch the cosine similarity decrease.
emb1 = model.encode("The large dog is red")
emb2 = model.encode("The big canine is red")
cos_sim = util.cos_sim(emb1, emb2)
print("3 - Cosine-Similarity:", cos_sim.item())

emb1 = model.encode("The large dog is red")
emb2 = model.encode("The big cat is red")
cos_sim = util.cos_sim(emb1, emb2)
print("4 - Cosine-Similarity:", cos_sim.item())

emb1 = model.encode("The large dog is red")
emb2 = model.encode("The big house is red")
cos_sim = util.cos_sim(emb1, emb2)
print("5 - Cosine-Similarity:", cos_sim.item())

# unrelated sentences produce cosine similarity closer to zero.
emb1 = model.encode("The small rock is transparent")
emb2 = model.encode("Romeo, Romeo, wherefore are thou, Romeo")
cos_sim = util.cos_sim(emb1, emb2)
print("6 - Cosine-Similarity:", cos_sim.item())

Now we add the cosine_similarity calculation

In [ ]:
import weaviate
import cohere
from sentence_transformers import SentenceTransformer, util

def IsOkString(value):
    if not value or len(value)==0:
        return False 
    else:
        return True

print("Let's a go!")

print("Load the sentence transformer model: multi-qa-MiniLM-L6-cos-v1 ")
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')


print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

limit = 1
result_count = 0
aggregate_count = 0
batch_size = 3
class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

print("Creating cohere_client")
cohere_client = cohere.Client(api_key) # This is your trial API key


def wv_get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()


print("Have fun storming the castle!")

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = wv_get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        the_prompt = res['prompt']
        result_count += 1
        
        response = cohere_client.generate(
            model='command',
            prompt=the_prompt,
            max_tokens=300,
            temperature=0.9,
            k=0,
            stop_sequences=[],
            return_likelihoods='NONE')
                
        completion = response.generations[0].text.replace("\n", " ")
        vec_current_completion = model.encode(completion)
        cos_sim = util.cos_sim(res['_additional']['vector'], vec_current_completion)
        print("Cosine-Similarity:", cos_sim.item())
        print("------------------------------------------------------------------------------------------------")
    
    
    if(aggregate_count >= limit):
        break

    # Update the cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]

### Load the baseline samples into sqlite db
```sql
sqlite3 database: driftDb

INSERT INTO tSample (
                        chat_id,
                        sample_dt,
                        cos_sim,
                        completion
                    )
                    VALUES (
                        'chat_id',
                        'sample_dt',
                        'cos_sim',
                        'completion'
                    );
```

Set these as appropriate
5 and 10 is good for dev
```python
batch_size = 5  
limit = 10
```
20 and 1000 is good when ur ready to go
should only need 20 seconds.
```python
batch_size = 20  
limit = 1000
```

In [ ]:
import weaviate
import sqlite3

print("Connecting source_client to weaviate instance on localhost:8080...")
source_client = weaviate.Client("http://localhost:8080")
print("source_client created")

print("Connecting to sqlite driftDb")
DRIFT_DB_PATH = "..\\data\\driftDb.db"
con = sqlite3.connect(DRIFT_DB_PATH)
sql_cursor = con.cursor()

runtime = datetime.datetime.now().replace(second=0, microsecond=0)

# sql
def sql_insert_sample(chat_id, cos_sim, completion):
    sql = "INSERT INTO tSample (chat_id, run_time, cos_sim, completion ) VALUES ('{}', '{}', '{}', \"{}\");".format(chat_id, runtime, cos_sim, completion)
    print(sql)
    sql_cursor.execute(sql)
    con.commit()  
    
batch_size = 20  
limit = 1000
result_count = 0
aggregate_count = 0

class_name = "DriftBenchmark"
class_properties = ["prompt"]
cursor = None

#weaviate
def wv_get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):

    query = (
        client.query.get("DriftBenchmark", ["chat_id", "prompt", "completion"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()

# Batch import all objects to the target instance
while True:
    
    # From the SOURCE instance, get the next group of objects
    results = wv_get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, add the objects to the batch to be added to the target instance
    objects_list = results["data"]["Get"][class_name]
    aggregate_count += len(objects_list)
    
    print('aggregate_count = {}'.format(aggregate_count))
    
    for res in objects_list:
        # print(json.dumps(res, indent=4))
        print("{}. {}".format(result_count, res['completion']))
        # res["data"]["Get"]
        result_count += 1
        
        # insert benchmark baseline samples
        sql_insert_sample(res['chat_id'], 1.0, res['completion']) 
        
    if(aggregate_count >= limit):
        break

    # Update the wv cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]